In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
import talib

In [140]:
df = pd.read_csv("./nifty.csv")
df.head()

,Date,Open,High,Low,Close,volume,Turnover
0,12-JUN-2023,18595.05,18633.6,18559.75,18601.50,179463966.0,15386.04
1,13-JUN-2023,18631.80,18728.9,18631.80,18716.15,233167883.0,22612.62
2,14-JUN-2023,18744.60,18769.7,18690.00,18755.90,261354575.0,19520.08
3,15-JUN-2023,18774.45,18794.1,18669.05,18688.10,263004991.0,24820.77
4,16-JUN-2023,18723.30,18864.7,18710.50,18826.00,272771256.0,28877.67


# macd

In [105]:
data = ta.trend.MACD(df["Close"], 26, 12, 9)
df["ta_macd"]= data.macd()

In [106]:
macd_12 = df["Close"].ewm(span=12, adjust=False, min_periods=12).mean()
macd_26 = df["Close"].ewm(span=26, adjust=False, min_periods=26).mean()

df["calc_macd"] = macd_12 - macd_26

In [107]:
macd_26

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
           ...     
245    22557.661913
246    22611.920290
247    22659.866935
248    22704.680495
249    22750.478236
Name: Close, Length: 250, dtype: float64

In [108]:
df.iloc[27]

Date         20-JUL-2023
Open             19831.7
High            19991.85
Low              19758.4
Close           19979.15
volume       274698867.0
Turnover        30622.66
ta_macd       272.095894
calc_macd     272.095894
Name: 27, dtype: object

In [109]:
df[10:30]

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd
10,26-JUN-2023,18682.35,18722.05,18646.70,18691.20,171271325.0,17355.14,NaN,NaN
11,27-JUN-2023,18748.55,18829.25,18714.25,18817.40,222464605.0,21415.89,NaN,NaN
12,28-JUN-2023,18908.15,19011.25,18861.35,18972.10,398782019.0,37356.94,NaN,NaN
13,30-JUN-2023,19076.85,19201.70,19024.60,19189.05,247906587.0,26376.62,NaN,NaN
14,03-JUL-2023,19246.50,19345.10,19234.40,19322.55,226050219.0,22681.54,NaN,NaN
15,04-JUL-2023,19406.60,19434.15,19300.00,19389.00,226916635.0,26634.06,NaN,NaN
16,05-JUL-2023,19405.95,19421.60,19339.60,19398.50,224418797.0,24699.24,NaN,NaN
17,06-JUL-2023,19385.70,19512.20,19373.00,19497.30,268261285.0,28540.18,NaN,NaN
18,07-JUL-2023,19422.80,19523.60,19303.60,19331.80,237102566.0,23089.25,NaN,NaN
19,10-JUL-2023,19400.35,19435.85,19327.10,19355.90,268246132.0,25991.55,NaN,NaN


# rsi

In [78]:
def calculate_rsi(df, timePeriod):
    difference = df["Close"].diff()
    pos_diff = 0
    neg_diff = 0
    curr_avg_pos = 0
    curr_avg_neg = 0
    for i in range(1, len(difference)):
        diff = difference.iloc[i]
        if i<timePeriod:
            if diff>0:
                pos_diff += diff
            elif diff<0:
                neg_diff += abs(diff)
        if i == timePeriod -1:
            curr_avg_neg = neg_diff/timePeriod
            curr_avg_pos = pos_diff/timePeriod
            rs = curr_avg_pos/curr_avg_neg
            df.at[i, "calc_rsi"] = 100 - (100/(1+rs))
        elif i>=timePeriod:
            if diff>0:
                pos_diff = diff
                neg_diff = 0
            elif diff<0:
                pos_diff = 0
                neg_diff = abs(diff)
            else:
                pos_diff = 0
                neg_diff = 0


            curr_avg_neg = ((timePeriod -1)*curr_avg_neg + neg_diff)/timePeriod
            curr_avg_pos = ((timePeriod -1)*curr_avg_pos + pos_diff)/timePeriod

            rs = curr_avg_pos/curr_avg_neg
            df.at[i, "calc_rsi"] = 100 - (100/(1+rs))
    return
        
calculate_rsi(df, 14)

In [79]:
df.iloc[0:20]

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd,calc_rsi
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,179463966.0,15386.04,NaN,0.000000,NaN
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,233167883.0,22612.62,NaN,9.145869,NaN
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,261354575.0,19520.08,NaN,19.378150,NaN
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,263004991.0,24820.77,NaN,21.765513,NaN
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,272771256.0,28877.67,NaN,34.388484,NaN
5,19-JUN-2023,18873.30,18881.45,18719.15,18755.45,219815214.0,22699.84,NaN,38.258465,NaN
6,20-JUN-2023,18752.35,18839.70,18660.65,18816.70,211649257.0,19687.08,NaN,45.740547,NaN
7,21-JUN-2023,18849.40,18875.90,18794.85,18856.85,217481345.0,19102.48,NaN,54.284166,NaN
8,22-JUN-2023,18853.60,18886.60,18759.50,18771.25,252697871.0,24990.87,NaN,53.530772,NaN
9,23-JUN-2023,18741.85,18756.40,18647.10,18665.50,210562119.0,20619.60,NaN,43.894571,NaN


In [80]:
data = ta.momentum.RSIIndicator(df["Close"], 14)
df["ta_rsi"] = data.rsi()

In [81]:
df.iloc[0:20]

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd,calc_rsi,ta_rsi
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,179463966.0,15386.04,NaN,0.000000,NaN,NaN
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,233167883.0,22612.62,NaN,9.145869,NaN,NaN
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,261354575.0,19520.08,NaN,19.378150,NaN,NaN
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,263004991.0,24820.77,NaN,21.765513,NaN,NaN
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,272771256.0,28877.67,NaN,34.388484,NaN,NaN
5,19-JUN-2023,18873.30,18881.45,18719.15,18755.45,219815214.0,22699.84,NaN,38.258465,NaN,NaN
6,20-JUN-2023,18752.35,18839.70,18660.65,18816.70,211649257.0,19687.08,NaN,45.740547,NaN,NaN
7,21-JUN-2023,18849.40,18875.90,18794.85,18856.85,217481345.0,19102.48,NaN,54.284166,NaN,NaN
8,22-JUN-2023,18853.60,18886.60,18759.50,18771.25,252697871.0,24990.87,NaN,53.530772,NaN,NaN
9,23-JUN-2023,18741.85,18756.40,18647.10,18665.50,210562119.0,20619.60,NaN,43.894571,NaN,NaN


In [82]:
df["talib_rsi"] = talib.RSI(df["Close"], 13)

In [84]:
df.iloc[:-5]

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd,calc_rsi,ta_rsi,talib_rsi
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,1.794640e+08,15386.04,NaN,0.000000,NaN,NaN,NaN
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,2.331679e+08,22612.62,NaN,9.145869,NaN,NaN,NaN
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,2.613546e+08,19520.08,NaN,19.378150,NaN,NaN,NaN
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,2.630050e+08,24820.77,NaN,21.765513,NaN,NaN,NaN
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,2.727713e+08,28877.67,NaN,34.388484,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
240,30-MAY-2024,22617.45,22705.75,22417.00,22488.65,3.733745e+08,32991.46,112.893998,112.893998,48.925525,48.925525,48.391593
241,31-MAY-2024,22568.10,22653.75,22465.10,22530.70,5.721226e+08,67040.32,95.853438,95.853438,50.332978,50.332978,49.926417
242,03-JUN-2024,23337.90,23338.70,23062.30,23263.90,5.694256e+08,56418.25,139.899115,139.899115,67.269487,67.269487,67.937883
243,04-JUN-2024,23179.50,23179.50,21281.45,21884.50,1.006105e+09,93786.44,62.775937,62.775937,39.783505,39.783504,39.199941


# ema

In [110]:
data = ta.trend.EMAIndicator(df["Close"], 26)
df["ta_ema26"] = data.ema_indicator()

In [111]:
df["talib_ema26"] = talib.EMA(df["Close"], 26)

In [112]:
def calc_ema(df, period):
    weight = 2/(1+period)
    sma = np.mean(df[:period]["Close"])
    col_name = "calc_ema" + str(period)
    df.at[period-1, col_name] = sma

    for i in range(period, len(df)):
        df.at[i, col_name] = (df.iloc[i]["Close"]*weight) + (df.iloc[i-1][col_name]*(1-weight))

calc_ema(df, 26)

In [114]:
df["panda_ema26"] = df["Close"].ewm(span=26, adjust=False, min_periods=5).mean()

In [115]:
df

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd,ta_ema26,talib_ema26,calc_ema26,panda_ema26
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,179463966.0,15386.04,NaN,NaN,NaN,NaN,NaN,NaN
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,233167883.0,22612.62,NaN,NaN,NaN,NaN,NaN,NaN
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,261354575.0,19520.08,NaN,NaN,NaN,NaN,NaN,NaN
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,263004991.0,24820.77,NaN,NaN,NaN,NaN,NaN,NaN
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,272771256.0,28877.67,NaN,NaN,NaN,NaN,NaN,18640.616383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,06-JUN-2024,22798.60,22910.15,22642.60,22821.40,480413289.0,49047.39,73.776168,73.776168,22557.661913,22557.661909,22557.661909,22557.661913
246,07-JUN-2024,22821.85,23320.20,22789.05,23290.15,473554898.0,47621.54,120.858086,120.858086,22611.920290,22611.920286,22611.920286,22611.920290
247,10-JUN-2024,23319.15,23411.90,23227.15,23259.20,304392368.0,29060.40,153.899383,153.899383,22659.866935,22659.866931,22659.866931,22659.866935
248,11-JUN-2024,23283.75,23389.45,23206.65,23264.85,305185666.0,29190.91,178.483312,178.483312,22704.680495,22704.680492,22704.680492,22704.680495


# obv

In [117]:
data = ta.volume.OnBalanceVolumeIndicator(df["Close"], df["volume"])
df["ta_obv"]=data.on_balance_volume()

In [118]:
df["talib_obv"] = talib.OBV(df["Close"], df["volume"])

In [119]:
def calc_obv(df):
    col_name = "calc_obv"
    df.at[0, col_name] = 0
    for i in range(1, len(df)):
        if df.at[i, "Close"]>df.at[i-1, "Close"]:
            df.at[i, col_name] = df.at[i-1, col_name] + df.at[i, "volume"]
        elif df.at[i, "Close"]<df.at[i-1, "Close"]:
            df.at[i, col_name] = df.at[i-1, col_name] - df.at[i, "volume"]
        else:
            df.at[i, col_name] = df.at[i-1, col_name]

calc_obv(df)

In [120]:
df

,Date,Open,High,Low,Close,volume,Turnover,ta_macd,calc_macd,ta_ema26,talib_ema26,calc_ema26,panda_ema26,ta_obv,talib_obv,calc_obv
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,179463966.0,15386.04,NaN,NaN,NaN,NaN,NaN,NaN,1.794640e+08,179463966.0,0.0
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,233167883.0,22612.62,NaN,NaN,NaN,NaN,NaN,NaN,4.126318e+08,412631849.0,233167883.0
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,261354575.0,19520.08,NaN,NaN,NaN,NaN,NaN,NaN,6.739864e+08,673986424.0,494522458.0
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,263004991.0,24820.77,NaN,NaN,NaN,NaN,NaN,NaN,4.109814e+08,410981433.0,231517467.0
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,272771256.0,28877.67,NaN,NaN,NaN,NaN,NaN,18640.616383,6.837527e+08,683752689.0,504288723.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,06-JUN-2024,22798.60,22910.15,22642.60,22821.40,480413289.0,49047.39,73.776168,73.776168,22557.661913,22557.661909,22557.661909,22557.661913,1.308304e+10,NaN,NaN
246,07-JUN-2024,22821.85,23320.20,22789.05,23290.15,473554898.0,47621.54,120.858086,120.858086,22611.920290,22611.920286,22611.920286,22611.920290,1.355659e+10,NaN,NaN
247,10-JUN-2024,23319.15,23411.90,23227.15,23259.20,304392368.0,29060.40,153.899383,153.899383,22659.866935,22659.866931,22659.866931,22659.866935,1.325220e+10,NaN,NaN
248,11-JUN-2024,23283.75,23389.45,23206.65,23264.85,305185666.0,29190.91,178.483312,178.483312,22704.680495,22704.680492,22704.680492,22704.680495,1.355739e+10,NaN,NaN


# vwap

In [141]:
data = ta.volume.VolumeWeightedAveragePrice(df["High"], df["Low"], df["Close"], df["volume"], 1)
data.volume_weighted_average_price()

0      18598.283333
1      18692.283333
2      18738.533333
3      18717.083333
4      18800.400000
           ...     
245    22791.383333
246    23133.133333
247    23299.416667
248    23286.983333
249    23353.616667
Name: vwap_1, Length: 250, dtype: float64

In [144]:
(df['High'] + df['Low'] + df['Close']) / 3

0      18598.283333
1      18692.283333
2      18738.533333
3      18717.083333
4      18800.400000
           ...     
245    22791.383333
246    23133.133333
247    23299.416667
248    23286.983333
249    23353.616667
Length: 250, dtype: float64

In [142]:
def calc_vwap(df, period):
    col_name = "calc_vwap"
    data = (((df['High'] + df['Low'] + df['Close']) / 3) * df['volume']).cumsum() / df['volume'].cumsum()
    for i in range(period-1, len(data)):
        df.at[i, col_name] = sum(data[i-period+1:i+1])/period

calc_vwap(df, 1)

In [143]:
df

,Date,Open,High,Low,Close,volume,Turnover,calc_vwap
0,12-JUN-2023,18595.05,18633.60,18559.75,18601.50,179463966.0,15386.04,18598.283333
1,13-JUN-2023,18631.80,18728.90,18631.80,18716.15,233167883.0,22612.62,18651.400370
2,14-JUN-2023,18744.60,18769.70,18690.00,18755.90,261354575.0,19520.08,18685.188289
3,15-JUN-2023,18774.45,18794.10,18669.05,18688.10,263004991.0,24820.77,18694.140938
4,16-JUN-2023,18723.30,18864.70,18710.50,18826.00,272771256.0,28877.67,18718.099702
...,...,...,...,...,...,...,...,...
245,06-JUN-2024,22798.60,22910.15,22642.60,22821.40,480413289.0,49047.39,20956.694002
246,07-JUN-2024,22821.85,23320.20,22789.05,23290.15,473554898.0,47621.54,20970.857804
247,10-JUN-2024,23319.15,23411.90,23227.15,23259.20,304392368.0,29060.40,20980.557788
248,11-JUN-2024,23283.75,23389.45,23206.65,23264.85,305185666.0,29190.91,20990.150548
